In [2]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda as cuda
import os
import time
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import timm # Unofficial pytorch image models, for comparison
%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Use Nvidia GPU if available, for faster results 

ModuleNotFoundError: No module named 'torch'

In [3]:
# Define the transformations that will be applied to the images during the loading process
transform = transforms.Compose([
    transforms.Resize((640, 360)), # capture resolution / 2 for now, need to experiment with different sizes
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Mean and Std. Dev values used here are commonly used with the ImageNet dataset
])

NameError: name 'transforms' is not defined

In [ ]:
# Get Data
# We'll use a PyTorch Generator to make things repeatable (deterministic)
g = torch.Generator(device=device).manual_seed(2147483647)

# Load Data
dataset_path = './data'
dataset = datasets.ImageFolder(dataset_path, transform=transform) # Automatically assigns labels to examples based on the directory structure

# Generate 3 splits: Train (80%), Validation - Hyperparam selection (10%), Test (10%)
# May need to implement k-fold cross-validation since data is relatively small in size
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

train, val, test = torch.utils.data.random_split(dataset, [train_size, val_size, test_size], generator=g)

# Create Data Loaders for splits
batch_size = 32
train_dl = torch.utils.data.DataLoader(train, batch_size=batch_size)
val_dl = torch.utils.data.DataLoader(val, batch_size=batch_size)
test_dl = torch.utils.data.Dataloader(test, batch_size=batch_size)

In [ ]:
# Training Metrics
models = ["BasicCNN"] #, "VGG", "VisionTransformer", "DenseNet", "InceptionV4", "RegNet"]
#mtoi = {model: i for i, model in enumerate(models)}
times = {}
training_loss = {}
training_accuracy = {}
validation_loss = {}
validation_accuracy = {}

for model in models:
    times[model] = 0
    training_loss[model] = []
    training_accuracy[model] = 0
    validation_loss = []
    validation_accuracy = 0 


In [ ]:
# Define CNN 
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        
        # Define CNN layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size, stride)
        
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size, stride)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(in_features, out_features)
        self.relu3 = nn.Relu()
        
        self.fc2 = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        # Define the forward pass through the network
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = x.view(x.size(0), -1) # Flatten the tensor before passing to fully connected layers
        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

In [ ]:
model = BasicCNN()

# Hyperparameters
learning_rate = 0.1 # Learning rate -- Implement Learning Rate Decay Later 
criterion = nn.CrossEntropyLoss() # loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # In Future, experiment with ADAM, and LION optimizers
num_epochs = 5 

loss_list = []

# Training Time
start_event = cuda.Event(enable_timing=True)
end_event = cuda.Event(enable_timing=True)
# Begin clock
start_event.record()

# Train BasicCNN here
for epoch in range(num_epochs):
    for images, labels, in train_dl:
        optimizer.zero_grad() # Clear gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss_list.append(loss) 
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
# End clock
end_event.record()
cuda.synchronize() # Wait for GPU operations to complete

time_BasicCNN = start_event.elapsed_time(end_event) / 1000 # Convert to seconds
global times
times[BasicCNN] = time_BasicCNN

global training_loss
training_loss[BasicCNN].append(loss_list)

In [ ]:
# Define and train alternatives here


In [ ]:
# Plot comparisons of training statistics

In [ ]:
# Loss
global models
global training_loss
plt.figure(figsize=(20,12))
for model in models:
    for loss in training_loss[model]
        plt.plot(loss, label=f"{model}")
    
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Convergence Comparison")
plt.legend()

plt.show()

In [ ]:
# Training Accuracy
global models
global training_accuracy

acc_list = []
for model in models:
    acc_list.push(training_accuracy[model])

plt.figure(figsize=(20,12))
plt.bar(models, acc_list)
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison")
plt.show()

In [ ]:
# Validation Loss
global models
global validation_loss
plt.figure(figsize=(20,12))
for model in models:
    for loss in validation_loss[model]
        plt.plot(loss, label=f"{model}")
    
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Validation Loss Convergence Comparison")
plt.legend()

plt.show()

In [ ]:
# Validation Accuracy
global models
global validation_accuracy

acc_list = []
for model in models:
    acc_list.push(validation_accuracy[model])

plt.figure(figsize=(20,12))
plt.bar(models, acc_list)
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison of Different Models")
plt.show()

In [ ]:
# Learning Curve

In [ ]:
# Training Time
global models
global times

times_list = []
for model in models:
    times_list.push(times[model])
    
plt.figure(figsize(20,12))
plt.bar(models, times_list)
plt.xlabel("Models")
plt.ylabel("Training Time")
plt.title("Training Time Comparison of Different Models")
plt.show()